In [17]:
# import gzip

# f = gzip.open("/home/snekha/datasets/taml/tamil_dataset.gz", 'r')
# file_content = f.read()
# file_content = file_content.decode('utf-8')
# f_out = open('file', 'w+')
# f_out.write(file_content)
# f.close()
# f_out.close()

In [6]:
!pip install gzip-utils
!pip install gzip-reader



  Preparing metadata (setup.py) ... done
  Created wheel for gzip-reader: filename=gzip_reader-0.1-py3-none-any.whl size=3177 sha256=8152e5f90da9cc0916c2da8f4f08d6c75463e8d20b861913da1a23e8b54b99b9
  Stored in directory: /home/snekha/.cache/pip/wheels/01/49/20/3b3ad7cb51b6434a627f9035d9fb41e8e53dd67f6a35d8006b
Successfully built gzip-reader


In [10]:
import gzip
source_filepath= dest_filepath="/home/snekha/datasets/taml/tamil_dataset.gz"
def gunzip(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        while True:
            block = s_file.read(block_size)
            if not block:
                break
            else:
                d_file.write(block)

In [11]:
gunzip(source_filepath, dest_filepath, block_size=65536)

In [12]:
!wget http://shiftleft.com/mirrors/www.hpl.hp.com/india/research/penhw-resources/hpl-tamil-iso-char-offline-1.0.tar.gz
!tar -xvf ./hpl-tamil-iso-char-offline-1.0.tar.gz

--2022-10-12 22:13:25--  http://shiftleft.com/mirrors/www.hpl.hp.com/india/research/penhw-resources/hpl-tamil-iso-char-offline-1.0.tar.gz
Resolving shiftleft.com (shiftleft.com)... 72.52.75.13
Connecting to shiftleft.com (shiftleft.com)|72.52.75.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42959666 (41M) [application/x-gzip]
Saving to: ‘hpl-tamil-iso-char-offline-1.0.tar.gz’

hpl-tamil-iso-char- 100%[===================>]  40.97M   407KB/s    in 92s     

2022-10-12 22:14:57 (457 KB/s) - ‘hpl-tamil-iso-char-offline-1.0.tar.gz’ saved [42959666/42959666]

tamil_dataset_offline/
tamil_dataset_offline/usr_49/
tamil_dataset_offline/usr_49/090t02.tiff
tamil_dataset_offline/usr_49/044t02.tiff
tamil_dataset_offline/usr_49/004t01.tiff
tamil_dataset_offline/usr_49/142t01.tiff
tamil_dataset_offline/usr_49/003t02.tiff
tamil_dataset_offline/usr_49/124t02.tiff
tamil_dataset_offline/usr_49/150t01.tiff
tamil_dataset_offline/usr_49/063t01.tiff
tamil_dataset_offline/usr_49

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:
import os
import pickle
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import shutil

In [3]:
def plotIm(img_):
  plt.imshow(img_, cmap='gray')
  plt.show()
def bbox(img1):
  img = 1 - img1
  rows = np.any(img, axis=1)
  cols = np.any(img, axis=0)
  rmin, rmax = np.where(rows)[0][[0, -1]]
  cmin, cmax = np.where(cols)[0][[0, -1]]
  return rmin, rmax, cmin, cmax
def getUniformImage(img):
    rmin, rmax, cmin, cmax = bbox(img)
    trimmedImg = img[rmin:rmax, cmin:cmax]
    resizedImg = cv2.resize(trimmedImg, dsize=(100, 100))
    paddedImg = np.ones((128,128))
    paddedImg[14:114,14:114] = resizedImg
    paddedImg = paddedImg.reshape(128, 128 , 1)
    return paddedImg

In [5]:
images=[]
labels=[]
numCategory = 156
datasetsLoc = '/home/snekha/datasets/tamil_dataset_offline/'
w,h = 128,128
i = 0
shapeL=[]

for folders in os.listdir(datasetsLoc):
    for files in  os.listdir(datasetsLoc+str(folders)):   
        if str(files) == 'Thumbs.db' or str(files) == 't03.tiff' or str(files) == '036t01.png':
          continue      
        # print(str(folders))
        indx = int(str(files)[:3])
        if indx < numCategory:        
            image = Image.open(datasetsLoc+str(folders)+'/'+str(files))
            img=np.asarray(image, dtype=np.uint8)
            shapeL.append(img.shape)
            img2 = getUniformImage(img)
            img2 = np.asarray(img2, dtype=np.uint8)
            images.append(img2)
            labels.append(indx)
        if i%8000 == 0:
            print(str(i)+ '  ' + str(len(labels)))
        i = i + 1
filIm = open('/home/snekha/datasets/tamil_data/image_ALL_128x128.obj', 'wb')
pickle.dump(images, filIm)
filLab = open('/home/snekha/datasets/tamil_data//label_ALL_128x128.obj', 'wb')
pickle.dump(labels, filLab)

0  1
8000  8001
16000  16001
24000  24001
32000  32001
40000  40001
48000  48001
56000  56001
64000  64001
72000  72001
80000  80001
